# Prediction #

After a first check on the data, using ordinal regressions and linear regressions, we can add new variables we've seen can help with prediction and use them with more complex models such as tree based or mlp models.

In [1]:
import pandas as pd
import numpy as np
from sklearn import feature_selection
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn import datasets, ensemble
from sklearn import metrics
from sklearn.metrics import mean_squared_error,roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression,LinearRegression, PoissonRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, IsolationForest
from xgboost import XGBClassifier, XGBRegressor
import pickle
from difflib import SequenceMatcher
import seaborn as sns

In [2]:
def third_largest(row):
    sorted_row = sorted(row, reverse=True)
    return sorted_row[2] if len(sorted_row) >= 3 else None

def third_smallest(row):
    sorted_row = sorted(row)
    return sorted_row[2] if len(sorted_row) >= 3 else None

def get_data_cols(df, att, prefix):
    '''
    Gets an attribute, and adds columns to show mean, max, min and sd (ignoring zeros) for the first num_players players.

    Parameters:
    df : The dataframe
    att: which attribute (e.g., Weight)
    prefix: HomePlayer or AwayPlayer
    '''

    player_weight_cols = [col for col in df.columns if col.startswith(f"{prefix}") and col.endswith(f"_{att}")]
    att = att.replace('(', ' ')
    if not player_weight_cols:
        print('no col with', att)
        return df

    # Select only the columns corresponding to the first num_players players
    player_weight_cols_subset = player_weight_cols[:11]  # Selecting the first 11 players

    # Replace zeros with NaN
    df[player_weight_cols_subset] = df[player_weight_cols_subset].replace(0, np.nan)


    if(att == 'Overall'):
        df[f"{prefix}_{att}_max"] = df[player_weight_cols_subset].max(axis=1, skipna=True)
        df[f"{prefix}_{att}_min"] = df[player_weight_cols_subset].min(axis=1, skipna=True)
        df[f"{prefix}_{att}_sd"] = df[player_weight_cols_subset].std(axis=1, skipna=True)
        df[f"{prefix}_{att}_mean"] = df[player_weight_cols_subset].mean(axis=1, skipna=True)
        df[f"{prefix}_{att}_mean_ln"] = np.log(df[player_weight_cols_subset]).mean(axis=1, skipna=True)
        df[f"{prefix}_{att}_mean_sqrt"] = np.sqrt(df[player_weight_cols_subset]).mean(axis=1, skipna=True)
        df[f"{prefix}_3rd_best"] = df[player_weight_cols_subset].apply(third_largest, axis=1)
        df[f"{prefix}_3rd_worst"] = df[player_weight_cols_subset].apply(third_smallest, axis=1)


  
    if(' ' in att): # 2 worded attributes so instead of dribbling total we will have dribbling
        df[f"{prefix}_{att.split()[0]}_mean"] = df[player_weight_cols_subset].mean(axis=1, skipna=True)

    else:
        df[f"{prefix}_{att}_mean"] = df[player_weight_cols_subset].mean(axis=1, skipna=True)


    return df
def find_most_similar_name(target_name, names_list, threshold=0.25):
    similarities = [(other_name, SequenceMatcher(None, target_name, other_name).ratio()) for other_name in names_list]
    if (len(similarities) == 0):
        return None, None
    most_similar_name, similarity_score = max(similarities, key=lambda x: x[1])
    
    if similarity_score >= threshold:
        #print('found player '+most_similar_name+" with a score of "+str(similarity_score))
        return most_similar_name, similarity_score
    else:
        print("didn't find", target_name)
        return None, None
def find_player(player_name, club_name,  df, attributes):
    #first we filter by club name
    temp = df[df['Club Name'] == club_name]

    #Now we find the player
    sim_name, full_score = find_most_similar_name(player_name, temp['Full Name'])
    sim_nickname, short_score = find_most_similar_name(player_name, temp['Known As'])
    if sim_name: # there is a full name
        ## if there is a nickname we have to check
        if sim_nickname and full_score > short_score:
            print('found plauyer',sim_name)
            return temp[temp['Full Name'] == sim_name].iloc[0][attributes]
            
        elif sim_nickname:
            print('found plauyer',sim_nickname)
            return temp[temp['Known As'] == sim_nickname].iloc[0][attributes]
        print('found plauyer',sim_name)

        return temp[temp['Full Name'] == sim_name].iloc[0][attributes]
        
    if sim_nickname: # if there is a nickname but no full name
        print('found plauyer',sim_nickname)
        return temp[temp['Known As'] == sim_nickname].iloc[0][attributes]

    print("didn't find", player_name)
    return None


def ratings_col(df, att_df):
    '''
    puts for every player their 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'
    for each player i we will have the column
    HomePlayeri (if the player is on the home team)
    AwayPlayeri (if the player is on the away team)
    and we will use the home_team_name or away_team_name and the full name as the key (we will use find_player(player_name, club_name,  df, attributes))
    '''

    attributes = [ 'Overall']
    # Iterate through each player column
    for i in range(1, 12):

        home_col = f'HomePlayer{i}'
        away_col = f'AwayPlayer{i}'
        if home_col in df.columns or away_col in df.columns:
            # Add columns for home team players
            for att in attributes:
                
                df[home_col + "_" + att] = df.apply(
                    lambda row: (find_player(row[home_col], row['home_team_name'], att_df, attributes)[att]
                                 if isinstance(row[home_col], str) and find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None 
                                 else row[home_col]) if not pd.isna(row[home_col]) else 0,
                    axis=1
                )

            for att in attributes:
                df[away_col + "_" + att] = df.apply(
                    lambda row: (find_player(row[away_col], row['away_team_name'], att_df, attributes)[att]
                                 if isinstance(row[away_col], str) and find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None 
                                 else row[away_col]) if not pd.isna(row[away_col]) else 0,
                    axis=1
                )
def replace_nas(df):
    '''
    gets df, goes to the Age column and replaces None with 18, and replaces Weight(in kg) and Height(in cm) with the mean, and overall with the min value
    '''

    # 'Dribbling Total', 'Pace Total', 'Defending Total', 'Shooting Total', 'Physicality Total', 'Passing Total' our new fatures
    attributes = ['Overall']
    # First, deal with the None values
    for i in range(1,12):
        home_col = f'HomePlayer{i}'
        away_col = f'AwayPlayer{i}'
        if home_col in df.columns:
            #
            for att  in attributes: 
                non_zero_col = df[home_col+'_'+att].replace(0, None)
                df[home_col+'_'+att].fillna(non_zero_col.min())
        if away_col in df.columns:

            for att  in attributes: 
                non_zero_col = df[away_col+'_'+att].replace(0, None)
                df[away_col+'_'+att].fillna(non_zero_col.min())
            # df[home_col+"_Age"].fillna(18, inplace=True)
            
            # # dealing with some ratings

            # df[home_col+"_Total"].fillna(18, inplace=True)
            # df[home_col+"_Dribbling Total"].fillna(df[home_col+"_Dribbling Total"].replace(0,None).min(), inplace=True)
            # df[home_col+"_Defending Total"].fillna(df[home_col+"_Defending Total"].replace(0,None).min(), inplace=True)
            # df[home_col+"_Shooting Total"].fillna(df[home_col+"_Shooting Total"].replace(0,None).min(), inplace=True)
            # df[home_col+"_Physicality Total"].fillna(df[home_col+"_Physicality Total"].replace(0,None).min(), inplace=True)
            # df[home_col+"_Pace Total"].fillna(df[home_col+"_Pace Total"].replace(0,None).min(), inplace=True)

            # # Build assumption (ignoring zeros)
            # non_zero_weights = df[home_col+"_Weight(in kg)"].replace(0, None)
            # non_zero_heights = df[home_col+"_Height(in cm)"].replace(0, None)
            # non_zero_overall = df[home_col+"_Overall"].replace(0, None)
            # df[home_col+"_Weight"].fillna(non_zero_weights.mean(), inplace=True)
            # df[home_col+"_Height"].fillna(non_zero_heights.mean(), inplace=True)
            
            
            # # Ratings assumption
            # df[home_col+"_Overall"].fillna(non_zero_overall.min(), inplace=True)
            
            # # And for away
            # # Age assumption
            # df[away_col+"_Age"].fillna(18, inplace=True)

            # df[away_col+"_Total"].fillna(18, inplace=True)
            # df[away_col+"_Dribbling Total"].fillna(df[away_col+"_Dribbling Total"].replace(0,None).min(), inplace=True)
            # df[away_col+"_Defending Total"].fillna(df[away_col+"_Defending Total"].replace(0,None).min(), inplace=True)
            # df[away_col+"_Shooting Total"].fillna(df[away_col+"_Shooting Total"].replace(0,None).min(), inplace=True)
            # df[away_col+"_Physicality Total"].fillna(df[away_col+"_Physicality Total"].replace(0,None).min(), inplace=True)
            # df[away_col+"_Pace Total"].fillna(df[away_col+"_Pace Total"].replace(0,None).min(), inplace=True)

            # # Build assumption (ignoring zeros)
            # non_zero_weights_away = df[away_col+"_Weight"].replace(0, None)
            # non_zero_heights_away = df[away_col+"_Height"].replace(0, None)
            # non_zero_overall_away = df[away_col+"_Overall"].replace(0, None)
            # df[away_col+"_Weight"].fillna(non_zero_weights_away.mean(), inplace=True)
            # df[away_col+"_Height"].fillna(non_zero_heights_away.mean(), inplace=True)
            
            # # Ratings assumption
            # df[away_col+"_Overall"].fillna(non_zero_overall_away.min(), inplace=True)
            

    # Dealing with zero values
    for i in range(1, 12):
        home_col = f'HomePlayer{i}'
        away_col = f'AwayPlayer{i}'
        if home_col in df.columns:
            for att in attributes:
                df[home_col+"_"+att].replace(0, None, inplace=True)
        if away_col in df.columns:
            for att in attributes:
                df[away_col+"_"+att].replace(0, None, inplace=True)
     
def set_season_cols(columns, season):
    for c in season.columns:
        for att in ['Overall']:
            if att in c:
                columns.append(c)
    
def find_closest_name(name, other_names):
    checked = []
    for other_name in other_names:
        if other_name in checked:
            continue
        if(isinstance(other_name, float)):
            continue
        if name in other_name:
            return other_name

def pipeline(train_arr, fifa):
    train_df = pd.read_csv(train_arr, encoding='latin1')
    ratings_df = pd.read_csv(fifa,encoding='latin1')
    ratings_df.rename(columns={'long_name' : 'Full Name', 'height_cm':'Height(in cm)',
                          'weight_kg' : 'Weight(in kg)',
                          'age' : 'Age', 'club_name' :"Club Name",
                          'Team' : 'Club Name',
                          'overall' : 'Overall', 'dribbling' : 'Dribbling Total',
                          'pace' : 'Pace Total', 'defending' : "Defending Total" ,
                          "shooting" : 'Shooting Total', 'physic' : 'Physicality Total',
                          'passing' : "Passing Total", 'short_name' : 'Known As'},inplace=True)
    print(ratings_df.columns)
    #first we replace the names with the numbers
    # fix_html_col(ratings_df, 'away_fromation')
    # fix_html_col(ratings_df, 'home_formation')

    # Define a dictionary
    fifa_team_names = ratings_df['Club Name'].values
    checked = []
    not_found = []
    names_fix = {'Celta Vigo': 'RC Celta de Vigo'
                 , 'Atlético Madrid': 'Atlético de Madrid'
                 , 'Bayern Munich': 'FC Bayern München'
                 , 'Eint Frankfurt': 'Eintracht Frankfurt'
                 , 'Gladbach': 'Borussia Mönchengladbach',
                 'Sporting Gijón':'Real Sporting de Gijón',
                 'Man Utd': "Manchester United",
             "Man City": 'Manchester City',
             "West Ham": "West Ham United",
             "Nott'm Forest": "Nottingham Forest",
             'Spurs': "Tottenham Hotspur",
             'Wolves': "Wolverhampton Wanderers",
             "Brighton and Hove Albion": "Brighton & Hove Albion",
             "Bournemouth": "AFC Bournemouth",
             'Newcastle': 'Newcastle United',
             'Leicester': 'Leicester City',
             'Leeds': "Leeds United",
             'Huddersfield' : 'Huddersfield Town',
             'Swansea' : 'Swansea City',
             'Cardiff': 'Cardiff City',
             'Norwich' : 'Norwich City',
             'Stoke' : 'Stoke City',
             'West Brom' : 'West Bromwich Albion',
             'Hull' : 'Hull City',
             'QPR' : 'Queens Park Rangers',
             'Sheffield Utd' : 'Sheffield United'}
    
    for team in train_df['home_team_name'].values:
        
        if (team not in fifa_team_names and team not in checked) and team not in names_fix.keys():
            checked.append(team)
            #print(team+" not found")
            
            alt = find_closest_name(team, fifa_team_names)
            if(alt):
                print(team+" can be replaced with "+alt)
                names_fix[team] = alt
            else:
                not_found.append(team)

    print(not_found)
    for key in names_fix.keys():
        ratings_df['Club Name'].replace(names_fix[key],key, inplace=True)    
 
    print("Finished Loading test")
    # print(train_df.columns)
    att_list = ['Overall']
    numerical_cols = []
    statistic_list = ['mean', 'min', 'max', 'sd', '3rd_best', '3rd_worst']
    #statistic_list = ['mean' ,'sd']
    cat_cols=['home_team_name', 'away_team_name']

    
    att_list = ['Overall']
    for att in att_list:
        if att not in ratings_df.columns:
            print('fix',att)
    
    # Sanity Check
    ratings_df = ratings_df[['Full Name', 'Club Name', 'Known As']+ att_list]
    ratings_col(train_df, ratings_df)

    replace_nas(train_df)
    train_df['Matchweek'] = train_df['Matchweek'].str.split(' ').str[1].astype(int)

    final_cols = ['league', 'home_score', 'home_team_name', 'away_score', 'away_team_name', 'home_GD_prior', 'away_GD_prior', 'home_Points_prior', 'away_Points_prior']
    set_season_cols(final_cols, train_df)

    
    #binary_cols = ['Home_Adv_Team', 'Strong_Away'] ## From ar
    

    for att in att_list:
        get_data_cols(train_df, att, prefix='HomePlayer')
        get_data_cols(train_df, att, prefix='AwayPlayer')
    
    for col in train_df.columns:
        for s in statistic_list:
            if s in col:
                numerical_cols.append(col)

    cat_cols=['Unnamed: 0', 'home_team_name', 'away_team_name', 'home_GD_prior', 
              'home_Points_prior', 'home_GD_form', 'home_Points_form', 
                'away_GD_prior', 'away_Points_prior', 'away_GD_form', 
                'away_Points_form','league', 'B365A', 'B365D', 'B365H', 'Matchweek']
    print('finished pipeline')
    return train_df[numerical_cols + cat_cols]


In [3]:
from scipy.stats import skellam

def extract_poisson_probas(home_pred, away_pred):
    '''
    creates a probability matrix according to the regression

    Parameters:
    home_pred: expected home score
    away_pred: expected away score
    '''

    probability_matrix = np.zeros((len(home_pred), 3))  # 3 columns (-1, 0, 1 probabilities)
    
    for i in range(len(home_pred)):
        # Check for NaN values and skip the calculation if found
        if np.isnan(home_pred[i]) or np.isnan(away_pred[i]):
            continue

        home_rounded = home_pred[i]
        away_rounded = away_pred[i]
                
        probability_matrix[i, 0] = skellam.cdf(-1, home_rounded, away_rounded)
        probability_matrix[i, 1] = skellam.pmf(0, home_rounded, away_rounded)
        probability_matrix[i, 2] = skellam.sf(0, home_rounded, away_rounded)
        if(probability_matrix[i, 0] == np.nan or probability_matrix[i, 1] == np.nan or probability_matrix[i, 2] == np.nan):
            print("NAN PRODUCED IN INDEX", i)
        
    return probability_matrix

def extract_poisson_probas_binary(home_pred, away_pred):
    '''
    creates a probability matrix according to the regression

    Parameters:
    home_pred: expected home score
    away_pred: expected away score
    '''

    probability_matrix = np.zeros((len(home_pred), 2))  # 3 columns (-1, 0, 1 probabilities)
    
    for i in range(len(home_pred)):
        # Check for NaN values and skip the calculation if found
        if np.isnan(home_pred[i]) or np.isnan(away_pred[i]):
            continue

        home_rounded = home_pred[i]
        away_rounded = away_pred[i]
                
        probability_matrix[i, 0] = skellam.cdf(-1, home_rounded, away_rounded)
        probability_matrix[i, 1] = skellam.sf(-1, home_rounded, away_rounded)
        if(probability_matrix[i, 0] == np.nan or probability_matrix[i, 1] == np.nan):
            print("NAN PRODUCED IN INDEX", i)
        
    return probability_matrix

In [4]:
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import poisson
def get_lineups_and_predict(match, fifa, no_home_adv = False):
    train = pipeline(match, fifa)

    #when using random forest

    example = pd.read_csv("X_test.csv")
    cols = example.columns
    home_reg = RandomForestRegressor()
    away_reg = RandomForestRegressor()
    print(cols)

    with open('encoder', 'rb') as file:
        enc = pickle.load(file)
    with open('RFAway.pkl', 'rb') as x:
        away_reg = pickle.load(x)
    with open('RFHome.pkl', 'rb') as x:
        home_reg = pickle.load(x)
    print("loaded models and encoder")
    # home_reg = XGBRegressor()
    # home_reg.load_model("XGBHome.json")
    # features = home_reg.get_booster().feature_names
    # away_reg = XGBRegressor()
    # away_reg.load_model("XGBAway.json")
    fixtures = list(train['home_team_name'] + ' V '+train['away_team_name'])

    print(fixtures)
    #pre processing
    #train.drop(['home_formation', 'away_fromation'], axis=1, inplace=True)
    train["Home_min_max"] = train['HomePlayer_Overall_max'] * train['HomePlayer_Overall_min']
    train['Away_min_max'] = train['AwayPlayer_Overall_max'] * train['AwayPlayer_Overall_min']
    train['HomelogOVR'] = np.log(train['HomePlayer_Overall_mean'])
    train['AwaylogOVR'] = np.log(train['AwayPlayer_Overall_mean'])
    # One-hot encode 'home_team_name'
    encoded = enc.transform(train[['home_team_name', 'away_team_name']])
    encoded_df = pd.DataFrame(encoded, columns=enc.get_feature_names_out(['home_team_name', 'away_team_name']))
    train = pd.concat([encoded_df, train], axis=1)

    train.drop(['home_team_name', 'away_team_name'], axis=1, inplace=True)
    print('finished encoding')
#    train = train[features]
    features = cols
    train = train[features]
    train.drop('Unnamed: 0', axis=1,inplace=True) # when using rf

    for col in train.columns:
        train[col] = train[col].astype(float)

    #train = train[away_reg.get_booster().feature_names]
    home_pred = home_reg.predict(train)
    away_pred = away_reg.predict(train)
    if no_home_adv:
        canon = train.copy()
        for col in train.columns:
            if "Home" in col:
                canon['Away'+col[4:]] = train[col]
            if "Away" in col:
                canon['Home'+col[4:]] = train[col]
            if 'home' in col:
                canon['away'+col[4:]] = train[col]
            if 'away' in col:
                canon['home'+col[4:]] = train[col]
        canon['B365H'] = train['B365A']
        canon['B365A'] = train['B365H']
        canon=canon[train.columns]
        home_pred += home_reg.predict(canon)
        home_pred/=2
        away_pred += away_reg.predict(canon)
        away_pred/=2

    home_chances = np.zeros(len(home_pred))
    away_chances = np.zeros(len(home_pred))
    for i in range(len(home_pred)):
        home_score = poisson(home_pred[i])
        away_score = poisson(away_pred[i])
        home_chances[i] = 1 - home_score.pmf(0)
        away_chances[i] = 1 - away_score.pmf(0)

    print("Home expected goals:", home_pred)
    print("Away expected goals:", away_pred)
    # add np arrays of proba to score
    probas = extract_poisson_probas_binary(home_pred, away_pred)
    print('Expected outcomes:')
    outcomes = probas * np.asarray(train[['B365A',  'B365H']])
    print("win outcomes", outcomes[:,1])
    print("loss / draw outcomes", outcomes[:,0])
    df = pd.DataFrame({'Fixture' : fixtures,
                       "Home Expected Goals": home_pred,
                       "Away Expected Goals": away_pred,
                       'Home Expected' : outcomes[:, 1],
                       'Away Expected' : outcomes[:, 0],
                       "Home Proba" : probas[:, 1],
                       "Away/draw Proba" : probas[:, 0],
                       "Home Chance to score" : home_chances,
                       "Away Chance to score" : away_chances,
                       "Best bet by sqrt (0 - away, 1 - draw, 2 - home team)": np.argmax(probas*np.sqrt(np.asarray(train[['B365A',  'B365H']])), 
                       axis = 1),
                       "Best bet by ln  (0 - away, 1 - draw, 2 - home team)": np.argmax(probas*np.log(np.asarray(train[['B365A', 'B365H']])), 
                       axis = 1),
                       "Best bet by sqrt -1  (0 - away, 1 - draw, 2 - home team)": np.argmax(probas*np.sqrt(np.asarray(train[['B365A', 'B365H']]) -1), 
                       axis = 1)})
    df.to_csv("predictions.csv")
    return probas

    

In [5]:
get_lineups_and_predict('format.csv', 'fifa_season.csv', no_home_adv=True)

<ipython-input-2-51712b4d4939>:219: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df = pd.read_csv(fifa,encoding='latin1')


Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'update_as_of', 'Known As', 'Full Name', 'player_positions', 'Overall',
       'potential',
       ...
       'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'],
      dtype='object', length=109)
[]
Finished Loading test
finished pipeline
Index(['Unnamed: 0', 'HomePlayer_Overall_sd', 'HomePlayer_Overall_mean_ln',
       'AwayPlayer_Overall_sd', 'AwayPlayer_Overall_mean_ln', 'home_GD_form',
       'home_Points_form', 'away_GD_form', 'away_Points_form', 'B365A',
       'B365D', 'B365H'],
      dtype='object')
loaded models and encoder
['Manchester City V Arsenal']


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- league
